# Find the Shortest Superstring

Given an array of strings words, return the smallest string that contains each string in words as a substring. If there are multiple valid strings of the smallest length, return any of them.

You may assume that no string in words is a substring of another string in words.

**Example 1:**

Input: words = ["alex","loves","leetcode"]
Output: "alexlovesleetcode"
Explanation: All permutations of "alex","loves","leetcode" would also be accepted.

**Example 2:**

Input: words = ["catg","ctaagt","gcta","ttca","atgcatc"]
Output: "gctaagttcatgcatc"
 

**Constraints:**

- 1 <= words.length <= 12
- 1 <= words[i].length <= 20
- words[i] consists of lowercase English letters.
- All the strings of words are unique

In [1]:
from functools import lru_cache

def overlap(a, b):
    """Returns the maximum overlap between the end of a and the start of b."""
    max_overlap = 0
    for i in range(1, min(len(a), len(b)) + 1):
        if a[-i:] == b[:i]:
            max_overlap = i
    return max_overlap

def shortestSuperstring(words):
    n = len(words)
    # Precompute all overlaps between each pair of words.
    overlaps = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j:
                overlaps[i][j] = overlap(words[i], words[j])

    # dp[mask][i] will hold the shortest superstring that contains all words in mask
    # and ends with the i-th word.
    dp = [[""] * n for _ in range(1 << n)]
    parent = [[-1] * n for _ in range(1 << n)]
    
    # Initialize dp for each single word (mask with just one word).
    for i in range(n):
        dp[1 << i][i] = words[i]

    # Iterate over all possible masks and all possible last words.
    for mask in range(1 << n):
        for i in range(n):
            if mask & (1 << i):  # If i-th word is in the current mask.
                prev_mask = mask ^ (1 << i)  # Mask without the i-th word.
                for j in range(n):
                    if prev_mask & (1 << j):  # If j-th word is in the previous mask.
                        candidate = dp[prev_mask][j] + words[i][overlaps[j][i]:]
                        if dp[mask][i] == "" or len(candidate) < len(dp[mask][i]):
                            dp[mask][i] = candidate
                            parent[mask][i] = j

    # Find the shortest superstring by looking at the last mask (all words included).
    final_mask = (1 << n) - 1
    min_superstring = None
    last_word = -1
    for i in range(n):
        if min_superstring is None or len(dp[final_mask][i]) < len(min_superstring):
            min_superstring = dp[final_mask][i]
            last_word = i

    # Reconstruct the path of words used to form the superstring.
    result = []
    mask = final_mask
    while last_word != -1:
        result.append(words[last_word])
        next_last_word = parent[mask][last_word]
        mask ^= (1 << last_word)
        last_word = next_last_word

    # The result is formed by reversing the path since we backtracked.
    result.reverse()
    
    return min_superstring

# Example 1
words1 = ["alex", "loves", "leetcode"]
print(shortestSuperstring(words1))  # Output: "alexlovesleetcode"

# Example 2
words2 = ["catg", "ctaagt", "gcta", "ttca", "atgcatc"]
print(shortestSuperstring(words2))  # Output: "gctaagttcatgcatc"

leetcodelovesalex
gctaagttcatgcatc
